## Low-Level Arrays


An array is a group of related variables stored one after another in a contiguous portion of computer memory. We can use a string as an example, which Python stores as a sequence of individual characters.

The benefit of using this type of data structure is the ability to retrieve any information in constant time (O(1)). By knowing the memory location at which the array starts and the position of the desired information, you can simply calculate the memory address using the formula `start + (index * element size)` (Of course, the arithmetic for calculating memory addresses within an array can
be handled automatically).

![String Example](../img/arrays/string_example.png)
*Example of how a string is stored in memory as an array of characters.*

### Referential Arrays

referential arrays store the bits that represents the reference address memory of the data. 


In scenarios where we don't know the exact memory requirements for the variables we want to store in an array, allocating maximum memory for each variable would be inefficient and wasteful. Instead, we can utilize referential arrays, where we store references to the memory addresses where these variables are stored.

![Referential Array Example](../img/arrays/string_reference_array.png)
*Example illustrating a referential array storing references to strings.*

### Compact Arrays

compact arrays store the bits that represents the primary data. 

They have several advantages over referential structures in terms of computing performance, most significantly, the overall memory usage will be much lower for a compact structure because there is no overhead devoted to the explicit storage of the sequence of memory references.

![Compact Array Example](../img/arrays/string_compact_array.png)

*Example illustrating a compact array storing references to strings.*

#  Dynamic Arrays

When creating a low-level array in a computer system, the precise size of that array must be explicitly declared in order for the system to properly allocate a consecutive piece of memory for its storage.

The first key to providing the semantics of a dynamic array is that a dynamic array instance maintains an underlying array that often has greater capacity than the current length of the dynamic array. For example, if a user create an dynamic array with 10 elements, we can create an underlying array with 15 elements size (5 extra element sizes), than facilitating the user to append new itens to their dynamic array. 

If a user continues to append elements to a list, any reserved capacity will eventually be exhausted. In that case, the class requests a new, larger array from the system, and initializes the new array so that its prefix matches that of the existing smaller array.

### Code Fragment 5.1

An experiment to investigate the correlation between the length of a list and its underlying size in Python.

In this experiment, we observe that even an empty list consumes a certain amount of memory. Additionally, we note that the length of the list does not increase linearly as elements are added. Instead, the length only increases when attempting to insert more elements than the current capacity of the underlying array. At this point, a new underlying array is created with a larger capacity, and the elements are transferred to it. This process continues as more elements are added to the list.

In [1]:
#Code Fragment 5.1
import sys


data = []
for k in range(10):
    a = len(data)
    b = sys.getsizeof(data)
    print(f"Length: {a}; Size in bytes: {b}")
    data.append(None)
    

Length: 0; Size in bytes: 56
Length: 1; Size in bytes: 88
Length: 2; Size in bytes: 88
Length: 3; Size in bytes: 88
Length: 4; Size in bytes: 88
Length: 5; Size in bytes: 120
Length: 6; Size in bytes: 120
Length: 7; Size in bytes: 120
Length: 8; Size in bytes: 120
Length: 9; Size in bytes: 184
